# Analyze the activation of the Large Language Models (Llama-7B for an example.)

The inspiration of our proposed DynamicK-Tuning method is that the Pre-trained Large Language Models are sparsely activated, 

In [10]:
from transformers import LlamaForCausalLM, AutoTokenizer
model = LlamaForCausalLM.from_pretrained("./models/Llama-2-7b-hf/")
tokenizer = AutoTokenizer.from_pretrained("./models/Llama-2-7b-hf/")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.21it/s]
/home/hkustadmin/miniconda3/envs/emo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/hkustadmin/miniconda3/envs/emo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect paramete

In [26]:
# sample 100 dataset from the model
import random
from datasets import load_dataset
data = load_dataset("data/alpaca")['train']
data_idx_sample = random.sample([_ for _ in range(len(data))], 100)
data_samples = [data[i] for i in data_idx_sample]
print(data_samples)

[{'instruction': 'Given the following information, create a survey question.', 'input': 'Topic: Discount incentives', 'output': 'How likely would you be to make a purchase if you were offered discounts or incentives?', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven the following information, create a survey question.\n\n### Input:\nTopic: Discount incentives\n\n### Response:\nHow likely would you be to make a purchase if you were offered discounts or incentives?'}, {'instruction': 'Identify 5 key features of a project.', 'input': 'Project: Building a renewable energy plant', 'output': 'The key features for building a renewable energy plant would include selecting a suitable site, securing land rights, securing financing and permissions, selecting and implementing a renewable energy technology, and designing an appropriate energy management s

Register forward and the backward hooks of the FFN layer in Llama model

In [11]:
import torch
import torch.nn as nn

n_layer = 32
hidden_inp = {
    "gate_proj": [[] for _ in range(n_layer)], 
    "up_proj": [[] for _ in range(n_layer)],
    "down_proj": [[] for _ in range(n_layer)]
}
hidden_out = {
    "gate_proj": [[] for _ in range(n_layer)], 
    "up_proj": [[] for _ in range(n_layer)],
    "down_proj": [[] for _ in range(n_layer)]
}
def forward_input_hook(name, layer_id):
    def fn(module: nn.Module, inp, out):
        hidden_inp[name][layer_id].append(inp)
    return fn

def forward_output_hook(name, layer_id):
    def fn(module: nn.Module, input, output):
        hidden_out[name][layer_id].append(inp)
    return fn
            
for n, m in model.named_modules():
    if len(n.split(".")) < 5:
        continue
    layer_id = int(n.split(".")[2])
    name = n.split(".")[-1]
    if "LlamaMLP" in n.split("."):
        m.register_forward_hook(forward_input_hook(name, layer_id))
        m.register_forward_hook(forward_output_hook(name, layer_id))
            

In [12]:
# sample 100 examples from the dataset
from tqdm import tqdm
from utils import PROMPT_DICT
prompt = PROMPT_DICT["prompt_input"]
for d in tqdm(data_samples):
    inp = prompt.format_map(d)
    out = f"{example['output']}{tokenizer.eos_token}"
    text = inp + out
    input_ids = tokenizer(out, return_tensors='pt').input_ids
    y = model(input_ids)


In [ ]:
gate_out = hidden_out['gate_proj']
inter_h = hidden_inp['down_proj']

We 